In [8]:
from brian2 import * 
from brian2.parsing.sympytools import str_to_sympy 
from sympy import Matrix 
from brian2.units.fundamentalunits import DIMENSIONLESS

In [3]:
def get_sensitivity_equations(group, parameters):
    eqs = group.equations
    diff_eqs = eqs.get_substituted_expressions(group.variables)
    diff_eq_names = [name for name, _ in diff_eqs]

    system = Matrix([diff_eq[1] for diff_eq in diff_eqs])
    J = system.jacobian(diff_eq_names)

    sensitivity = []
    sensitivity_names = []
    for parameter in parameters:
        F = system.jacobian([parameter])
        names = ['S_{}_{}'.format(diff_eq_name, parameter)
                 for diff_eq_name in diff_eq_names]
        sensitivity.append(J * Matrix(names) + F)
        sensitivity_names.append(names)

    new_eqs = []
    for names, sensitivity_eqs, param in zip(sensitivity_names, sensitivity, parameters):
        for name, eq, orig_var in zip(names, sensitivity_eqs, diff_eq_names):
            unit = eqs[orig_var].dim / group.namespace[parameter].dim
            new_eqs.append('d{lhs}/dt = {rhs} : {unit}'.format(lhs=name,
                                                           rhs=eq,
                                                           unit=repr(unit) if unit is not DIMENSIONLESS else '1'))
    new_eqs = Equations('\n'.join(new_eqs))
    return new_eqs

In [5]:
# Parameters
area = 20000*umetre**2

In [4]:
# The model
eqs = Equations('''
dv/dt = (gl*(El-v)-
         g_na*(m*m*m)*h*(v-ENa)-
         g_kd*(n*n*n*n)*(v-EK))/Cm : volt
dm/dt = alpha_m*(1-m)-beta_m*m : 1
dn/dt = alpha_n*(1-n)-beta_n*n : 1
dh/dt = alpha_h*(1-h)-beta_h*h : 1
alpha_m = 0.32*(mV**-1)*(13*mV-v+VT)/
         (exp((13*mV-v+VT)/(4*mV))-1.)/ms : Hz
beta_m = 0.28*(mV**-1)*(v-VT-40*mV)/
        (exp((v-VT-40*mV)/(5*mV))-1)/ms : Hz
alpha_h = 0.128*exp((17*mV-v+VT)/(18*mV))/ms : Hz
beta_h = 4./(1+exp((40*mV-v+VT)/(5*mV)))/ms : Hz
alpha_n = 0.032*(mV**-1)*(15*mV-v+VT)/
         (exp((15*mV-v+VT)/(5*mV))-1.)/ms : Hz
beta_n = .5*exp((10*mV-v+VT)/(40*mV))/ms : Hz
''')

In [6]:
P = NeuronGroup(4000, model=eqs, threshold='v>-20*mV', refractory=3*ms,
                method='exponential_euler', namespace={'Cm': (1*ufarad*cm**-2) * area,
                                                       'gl': (5e-5*siemens*cm**-2) * area,
                                                       'El': -60*mV,
                                                       'EK': -90*mV,
                                                       'ENa': 50*mV,
                                                       'g_na': (100*msiemens*cm**-2) * area,
                                                       'g_kd': (30*msiemens*cm**-2) * area,
                                                       'VT':-63*mV
                                                       })


In [21]:
xx = get_sensitivity_equations(P, ['g_na', 'gl', 'g_kd', 'El', 'EK', 'ENa'])
# xx

In [15]:
new_eq = get_sensitivity_equations(P, ['g_na'])
type(new_eq)

brian2.equations.equations.Equations

In [16]:
new_eq

dS_v_g_na/dt = -S_h_g_na*g_na*m**3.0*(-ENa + v)/Cm - 3.0*S_m_g_na*g_na*h*m**2.0*(-ENa + v)/Cm - 4.0*S_n_g_na*g_kd*n**3.0*(-EK + v)/Cm + S_v_g_na*(-g_kd*n**4.0 - g_na*h*m**3.0 - gl)/Cm - h*m**3.0*(-ENa + v)/Cm : metre ** 4 * kilogram ** 2 * second ** -6 * amp ** -3
dS_m_g_na/dt = S_m_g_na*(-0.32*mV**(-1.0)*(VT + 13.0*mV - v)/(ms*(exp((0.25*VT + 3.25*mV - 0.25*v)/mV) - 1.0)) - 0.28*mV**(-1.0)*(-VT - 40.0*mV + v)/(ms*(exp((-0.2*VT - 8.0*mV + 0.2*v)/mV) - 1.0))) + S_v_g_na*(0.056*m*mV**(-2.0)*(exp((-0.2*VT - 8.0*mV + 0.2*v)/mV) - 1.0)**(-2.0)*(-VT - 40.0*mV + v)*exp((-0.2*VT - 8.0*mV + 0.2*v)/mV)/ms - 0.28*m*mV**(-1.0)/(ms*(exp((-0.2*VT - 8.0*mV + 0.2*v)/mV) - 1.0)) + 0.08*mV**(-2.0)*(1.0 - m)*(exp((0.25*VT + 3.25*mV - 0.25*v)/mV) - 1.0)**(-2.0)*(VT + 13.0*mV - v)*exp((0.25*VT + 3.25*mV - 0.25*v)/mV)/ms - 0.32*mV**(-1.0)*(1.0 - m)/(ms*(exp((0.25*VT + 3.25*mV - 0.25*v)/mV) - 1.0))) : ohm
dS_n_g_na/dt = S_n_g_na*(-0.032*mV**(-1.0)*(VT + 15.0*mV - v)/(ms*(exp((0.2*VT + 3.0*mV - 0.2*v)/mV) - 1